In [271]:
import pickle
import torch

In [272]:
folder="/data/home/umang/Trajectory_project/GPS-MTM/outputs/test_geolife/2025-08-27_10-47-20/test_outputs/random_masking_0.5_testing/RANDOM"
pred_pickle_path="{}/predictions_batch_6.pkl".format(folder)
batch_pickle_path = "{}/ground_truth_batch_6.pkl".format(folder)
attention_pickle_path = "{}/attention_masks_batch_6.pkl".format(folder)
masks_pickle_path = "{}/masks_batch_6.pkl".format(folder)
with open(pred_pickle_path, 'rb') as f:
    predictions = pickle.load(f)

with open(batch_pickle_path, 'rb') as f:
    batch = pickle.load(f)

with open(attention_pickle_path, 'rb') as f:
    attention_masks = pickle.load(f)

with open(masks_pickle_path, 'rb') as f:
    masks = pickle.load(f)

In [273]:
masks['states'].shape

torch.Size([2549])

In [274]:
predictions["states"].shape

torch.Size([1, 2549, 198])

In [275]:
predictions["actions"].shape

torch.Size([1, 2549, 11])

In [276]:
# get the most frequently top-5 most repeated in ground_truth_list_States
# #don't use torch top-k..
ground_truth_list_states = []
for i in range(len(attention_masks)):
     # first zero mask is first time when attention_masks[i] becomes zero
    zero_indices = (attention_masks[i].flatten() == 0).nonzero(as_tuple=True)[0]
    first_zero_mask = zero_indices[0].item() if len(zero_indices) > 0 else attention_masks[i].numel()
    
    ground_truth_states= batch["states"][i, :first_zero_mask, :]
    ground_truth_states = torch.argmax(ground_truth_states, dim=-1)
    ground_truth_list_states.append(ground_truth_states)

k=4
ground_truth_list_states = torch.cat(ground_truth_list_states).flatten()
unique, counts = torch.unique(ground_truth_list_states, return_counts=True)
top_k_ground_truth = unique[torch.topk(counts, k=k).indices]
top_k_ground_truth

top_1_ground_truth = unique[torch.topk(counts, k=1).indices]
top_4_ground_truth = unique[torch.topk(counts, k=4).indices]    
# top_5_ground_truth = unique[torch.topk(counts, k=5).indices]
# top_10_ground_truth = unique[torch.topk(counts, k=10).indices]
# top_20_ground_truth = unique[torch.topk(counts, k=20).indices]


In [277]:
batch['actions'][0]

tensor([[ 1.0000,  1.0000,  3.0000,  ...,  0.2430,  0.1561, -0.1035],
        [ 1.0000,  1.0000,  4.0000,  ...,  0.2349,  0.1568, -0.1035],
        [ 1.0000,  1.0000,  4.0000,  ...,  0.2351,  0.1564, -0.1035],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [278]:
masks

{'states': tensor([1, 0, 1,  ..., 1, 1, 1]),
 'actions': tensor([1, 0, 1,  ..., 1, 1, 1])}

In [279]:
predictions_list_states=[]
predictions_list_actions=[]
ground_truth_list_states=[]
ground_truth_list_actions=[]
correct_list=[]


# Get mask for this sequence
mask_states = masks["states"] == 1
mask_actions = masks["actions"] == 1

net_masks_states= attention_masks* masks["states"]
net_masks_actions= attention_masks* masks["actions"]

# Apply masks to get only positions where mask is 1
predictions_states = predictions["states"][net_masks_states.bool(), :]
predictions_actions = predictions["actions"][net_masks_actions.bool(), :]
ground_truth_states = batch["states"][net_masks_states.bool(), :]
ground_truth_actions = batch["actions"][net_masks_actions.bool(), :]

# take argmax of predictions states on last dimension

predictions_list_states.append(torch.argmax(predictions_states, dim=-1))
predictions_list_actions.append(torch.argmax(predictions_actions, dim=-1))
ground_truth_list_states.append(torch.argmax(ground_truth_states, dim=-1))
ground_truth_list_actions.append(torch.argmax(ground_truth_actions, dim=-1))

print(ground_truth_states)
correct = (predictions_states == ground_truth_states).float()
correct_list.append(correct)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [280]:
net_masks_states[..., 0:100]

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
         1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
         1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
         1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0.,
         1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
         1., 0., 1., 1., 1., 1., 1., 1., 1., 0.]])

In [281]:
predictions_states[0:5, :]

tensor([[-12.3103, -18.5630, -19.0916, -12.4936,   6.2452, -16.9787, -12.8909,
         -21.2029, -25.6687, -11.9445, -10.7326, -13.7131, -17.0386, -10.1652,
         -15.9799, -15.0842, -23.2234, -12.5690, -14.4104, -11.4185, -15.0160,
         -19.0948, -20.3912, -10.6393, -28.7788, -18.0792, -17.0579, -13.0886,
         -20.1356, -15.0936, -20.1976, -14.9964, -16.7160, -26.7419, -18.8920,
         -13.8157, -18.1270, -30.9393, -28.1965, -14.2142, -22.7878, -17.4191,
         -26.2110, -10.9417, -18.2254, -17.1808, -24.7699, -25.0637, -14.4463,
         -19.3941, -16.5795, -14.5899, -18.1267, -11.1659, -29.6646, -15.0895,
         -19.2787, -26.0587, -18.1465, -20.1063, -21.2770, -21.4669, -26.1188,
         -10.9925, -18.1611, -18.5843, -25.9701, -18.3013, -23.3829, -21.8736,
         -20.0076, -22.3631, -24.1364, -20.0005, -22.6927, -26.9765, -13.3929,
         -22.3499, -19.9872, -18.7133, -25.5618, -22.1150, -21.6509, -17.1083,
         -18.7899, -20.9000, -18.9588, -21.7372, -30

In [282]:
predictions_states.shape,ground_truth_states.shape

(torch.Size([121, 198]), torch.Size([121, 198]))

In [283]:
predictions_list_states=torch.cat(predictions_list_states).flatten()
ground_truth_list_states=torch.cat(ground_truth_list_states).flatten()
predictions_list_states.shape, ground_truth_list_states.shape

(torch.Size([121]), torch.Size([121]))

In [293]:
predictions_list_states[0:200], ground_truth_list_states[0:200]

(tensor([ 4,  9,  4,  4,  4, 32, 51, 52,  3,  4,  3,  3,  3,  3,  3,  3, 19, 34,
         19,  9, 20, 15,  9,  6,  3, 43,  4, 53,  4, 41,  4,  3,  4, 53,  9,  9,
         25, 27, 34, 41,  4,  4,  4, 25,  9, 25, 54,  4, 41, 13, 53, 53, 41, 15,
          4,  1, 31,  9,  4, 31,  9,  9, 15, 15, 17,  4,  4,  4,  4, 31, 10, 10,
         10, 10, 10, 10, 31, 31,  3,  4,  3, 31,  3,  9, 26, 10,  6, 41, 10, 10,
          3, 39, 41,  4,  0,  4, 40, 40, 40,  9,  4, 39, 13, 31,  4, 13,  4, 41,
         41, 34,  4, 34, 39,  9, 27,  9, 41, 39, 36, 17,  3]),
 tensor([ 4,  9,  4,  4,  4, 32, 51, 52,  3,  4,  3,  3,  3,  3,  3,  3, 19, 34,
         19,  9, 20, 15,  9,  6,  3, 43,  4, 53,  4, 41,  4,  3,  4, 53,  9,  9,
         25, 27, 34, 41,  4,  4,  4, 25,  9, 25, 54,  4, 41, 13, 53, 53, 41, 15,
          4,  1, 31,  9,  4, 31,  9,  9, 15, 15, 17,  4,  4,  4,  4, 31, 10, 10,
         10, 10, 10, 10, 31, 31,  3,  4,  3, 31,  3,  9, 26, 10,  6, 41, 10, 10,
          3, 39, 41,  4,  0,  4, 40, 40, 40,  

In [303]:
print(net_masks_states[..., 0:300])

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
         1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
         1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
         1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0.,
         1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
         1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
         1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1.,
         0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0.,
         0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0.,

In [304]:
predictions_list_states[0:200]-ground_truth_list_states[0:200]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

In [287]:
## Calculate accuracy USING predicted states and ground truth list states.
total_correct = 0
total_elements = 0
for pred_states, gt_states in zip(predictions_list_states, ground_truth_list_states):
    correct = (pred_states == gt_states).float()
    total_correct += correct.sum().item()
    total_elements += correct.numel()
accuracy = total_correct / total_elements if total_elements > 0 else 0
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 1.0000


In [288]:
# filter predictions and ground truth for each unique class
unique_classes = torch.unique(ground_truth_list_states)
filtered_predictions = {}
filtered_ground_truth = {}
for cls in unique_classes:
    mask = (ground_truth_list_states == cls)
    filtered_predictions[cls.item()] = predictions_list_states[mask]
    filtered_ground_truth[cls.item()] = ground_truth_list_states[mask]

# sort classes by frequency
class_counts = []
for cls in unique_classes:
    count = (ground_truth_list_states == cls).sum().item()
    class_counts.append((cls.item(), count))

# sort by count in descending order
class_counts.sort(key=lambda x: x[1], reverse=True)

# compute and print accuracy, precision, recall, and f1 score for each class sorted by frequency
print(f"{'Class':<5} {'Count':<6} {'Accuracy':<8} {'Precision':<9} {'Recall':<6} {'F1':<6}")
print("-" * 50)

for cls, count in class_counts:
    # Get predictions and ground truth for this class
    cls_predictions = filtered_predictions[cls]
    cls_ground_truth = filtered_ground_truth[cls]
    
    # Accuracy for this class
    accuracy = (cls_predictions == cls_ground_truth).float().mean().item()
    
    # For precision, recall, F1: need to consider this class vs all others
    # True positives: predicted this class and actually this class
    tp = ((predictions_list_states == cls) & (ground_truth_list_states == cls)).sum().item()
    
    # False positives: predicted this class but actually not this class
    fp = ((predictions_list_states == cls) & (ground_truth_list_states != cls)).sum().item()
    
    # False negatives: didn't predict this class but actually this class
    fn = ((predictions_list_states != cls) & (ground_truth_list_states == cls)).sum().item()
    
    # Calculate precision, recall, F1
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    print(f"{cls:<5} {count:<6} {accuracy:<8.4f} {precision:<9.4f} {recall:<6.4f} {f1:<6.4f}")

Class Count  Accuracy Precision Recall F1    
--------------------------------------------------
4     26     1.0000   1.0000    1.0000 1.0000
3     14     1.0000   1.0000    1.0000 1.0000
9     13     1.0000   1.0000    1.0000 1.0000
10    9      1.0000   1.0000    1.0000 1.0000
41    9      1.0000   1.0000    1.0000 1.0000
31    7      1.0000   1.0000    1.0000 1.0000
15    4      1.0000   1.0000    1.0000 1.0000
34    4      1.0000   1.0000    1.0000 1.0000
39    4      1.0000   1.0000    1.0000 1.0000
53    4      1.0000   1.0000    1.0000 1.0000
13    3      1.0000   1.0000    1.0000 1.0000
25    3      1.0000   1.0000    1.0000 1.0000
40    3      1.0000   1.0000    1.0000 1.0000
6     2      1.0000   1.0000    1.0000 1.0000
17    2      1.0000   1.0000    1.0000 1.0000
19    2      1.0000   1.0000    1.0000 1.0000
27    2      1.0000   1.0000    1.0000 1.0000
0     1      1.0000   1.0000    1.0000 1.0000
1     1      1.0000   1.0000    1.0000 1.0000
20    1      1.0000   1.0000 

In [289]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Convert to numpy for sklearn compatibility
gt_numpy = ground_truth_list_states.cpu().numpy()
pred_numpy = predictions_list_states.cpu().numpy()

# Overall metrics
overall_accuracy = accuracy_score(gt_numpy, pred_numpy)
macro_f1 = f1_score(gt_numpy, pred_numpy, average='macro')
weighted_f1 = f1_score(gt_numpy, pred_numpy, average='weighted')
macro_precision = precision_score(gt_numpy, pred_numpy, average='macro')
weighted_precision = precision_score(gt_numpy, pred_numpy, average='weighted')
macro_recall = recall_score(gt_numpy, pred_numpy, average='macro')
weighted_recall = recall_score(gt_numpy, pred_numpy, average='weighted')

print("Overall Performance Metrics:")
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Weighted Precision: {weighted_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")
print(f"Weighted Recall: {weighted_recall:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(gt_numpy, pred_numpy))

# Class-wise metrics accounting for imbalance
unique_classes = torch.unique(ground_truth_list_states)
class_counts = [(cls.item(), (ground_truth_list_states == cls).sum().item()) for cls in unique_classes]
class_counts.sort(key=lambda x: x[1], reverse=True)

print(f"\n{'Class':<5} {'Count':<6} {'Freq%':<6} {'Precision':<9} {'Recall':<6} {'F1':<6}")
print("-" * 50)

for cls, count in class_counts:
    freq_pct = (count / len(ground_truth_list_states)) * 100
    
    # Calculate metrics for this specific class
    cls_mask_gt = (gt_numpy == cls)
    cls_mask_pred = (pred_numpy == cls)
    
    tp = ((pred_numpy == cls) & (gt_numpy == cls)).sum()
    fp = ((pred_numpy == cls) & (gt_numpy != cls)).sum()
    fn = ((pred_numpy != cls) & (gt_numpy == cls)).sum()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    print(f"{cls:<5} {count:<6} {freq_pct:<6.1f} {precision:<9.4f} {recall:<6.4f} {f1:<6.4f}")

Overall Performance Metrics:
Overall Accuracy: 1.0000
Macro F1 Score: 1.0000
Weighted F1 Score: 1.0000
Macro Precision: 1.0000
Weighted Precision: 1.0000
Macro Recall: 1.0000
Weighted Recall: 1.0000

Detailed Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           3       1.00      1.00      1.00        14
           4       1.00      1.00      1.00        26
           6       1.00      1.00      1.00         2
           9       1.00      1.00      1.00        13
          10       1.00      1.00      1.00         9
          13       1.00      1.00      1.00         3
          15       1.00      1.00      1.00         4
          17       1.00      1.00      1.00         2
          19       1.00      1.00      1.00         2
          20       1.00      1.00      1.00         1
          25       1.00      1.00      1.00         3
          2

In [290]:
# get the number of correct predictions for classes in top_10_ground_truth
#top_k=[top_1_ground_truth, top_5_ground_truth, top_10_ground_truth, top_20_ground_truth]
top_k=[top_1_ground_truth, top_4_ground_truth]
for i in range(len(top_k)):
    filtered_gt = ground_truth_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]
    filtered_pred = predictions_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]

    # get the number of correct predictions and accuracy
    correct_predictions = (filtered_pred == filtered_gt).sum()
    accuracy = correct_predictions / filtered_gt.numel()
    print(f"Number of correct predictions in {top_k[i]} ground truth: {correct_predictions.item()}")
    print(f"Accuracy of correct predictions in {top_k[i]} ground truth: {accuracy.item()}")

Number of correct predictions in tensor([4]) ground truth: 26
Accuracy of correct predictions in tensor([4]) ground truth: 1.0
Number of correct predictions in tensor([ 4,  3,  9, 10]) ground truth: 62
Accuracy of correct predictions in tensor([ 4,  3,  9, 10]) ground truth: 1.0


### Proof that model is working properly below.

-0.0207 is the index for mask value to be filled in. Corresponding mask is shown below.

In [291]:
# (Pdb) x_sample[0,:,0]
# tensor([-0.9309, -2.4709, -0.9647, -2.5226, -0.9906, -2.6224, -1.0323, -2.6396,
#         -1.0287, -2.5965, -0.0207, -0.0207, -1.0252, -2.4983, -1.0098, -0.0207,
#         -1.0417, -2.3514, -0.0207, -0.0207, -0.0207, -2.3262, -0.9608, -2.4531,
#         -0.9609, -2.5430, -0.0207, -2.4288, -0.0207, -1.5484, -0.9251, -2.4177,
#         -0.0207, -0.0207, -0.8637, -2.3879, -0.8872, -0.0207, -0.9029, -2.3117,
#         -0.9281, -2.3534, -1.0126, -2.4598, -0.9165, -0.0207, -0.8610, -2.4906,
#         -0.0207, -2.5534, -1.0234, -0.8551, -2.4868, -0.8567, -2.4971, -0.9002,
#         -2.4162, -0.0207, -0.0207, -0.9232, -0.0207, -0.9814, -0.0207, -0.0207,
#         -2.3403, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207], device='cuda:0')

In [292]:
# (Pdb) masks['states']
# tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
#         1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
#         0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
#         1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
#         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
#         0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
#         1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
#         1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#         1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
#         1, 1, 0, 1, 1], device='cuda:0')